# W205 - Project 3
## Erin Werner - Summer 2020

I am a data scientist at a game development company. My latest mobile game has two events that I am interested in tracking: buy a sword & join a guild. Each has metadata characterstic of such events (i.e., sword type, guild name, etc).

This notebook will cover the end-to-end process for the data engineering pipeline. The overall process includes initially generating test data as well as generating a notebook-based report of simple event analytics.

We will use this notebook to present our queries and findings. This notebook can help others in our business that need to act on our recommendations.

### Create the environment.

mkdir ~/w205/project-3-etwernerMIDS/

cd ~/w205/project-3-etwernerMIDS/

cp ~/w205/course-content/14-Patterns-for-Data-Pipelines/docker-compose.yml .

docker-compose pull

cp ~/w205/course-content/14-Patterns-for-Data-Pipelines/*.py .

*Once the base files are copied, we will append additional event types to both the game_api and write_sword_stream python script files. These include purchase a sword, purchase a knife, join a guild, sell a sword, sell a knife, and leave a guild.*

### Spin up docker and start Flask.

docker-compose up -d

*Results:*
* *Creating network "project3etwernermids_default" with the default driver*
* *Creating project3etwernermids_cloudera_1*
* *Creating project3etwernermids_presto_1*
* *Creating project3etwernermids_mids_1*
* *Creating project3etwernermids_zookeeper_1*
* *Creating project3etwernermids_kafka_1*
* *Creating project3etwernermids_spark_1*

docker-compose exec mids env FLASK_APP=/w205/project-3-etwernerMIDS/game_api.py flask run --host 0.0.0.0

*Results:*

 * Serving Flask app "game_api"
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)

## Task 1: Instrument your API server to log events to Kafka

### Set up to watch Kafka.

*In a separate terminal, run this command twice to create a topic and start kafkacat. Running kafkacat without -e will allow it to run continuously.*

docker-compose exec mids kafkacat -C -b kafka:29092 -t gameevents -o beginning

## Task 2: Assemble a data pipeline to catch these events: use Spark streaming to filter select event types from Kafka, land them into HDFS/parquet to make them available for analysis using Presto

### Streaming.

*In a separate terminal, run this command to start streaming in events.*

docker-compose exec spark spark-submit /w205/project-3-etwernerMIDS/write_swords_stream.py

### Check results in Hadoop.

*In a separate terminal, we can check our results.*

docker-compose exec cloudera hadoop fs -ls /tmp/sword_purchases

*Results:*

* *Found 1 items*
* *drwxr-xr-x   - root supergroup          0 2020-07-30 17:24 /tmp/sword_purchases/_spark_metadata*

*The remaining commands will have similar results.*

docker-compose exec cloudera hadoop fs -ls /tmp/knife_purchases

docker-compose exec cloudera hadoop fs -ls /tmp/guilds_joined

docker-compose exec cloudera hadoop fs -ls /tmp/sword_sold

docker-compose exec cloudera hadoop fs -ls /tmp/knife_sold

docker-compose exec cloudera hadoop fs -ls /tmp/guild_left

### Set up Presto.

docker-compose exec cloudera hive

### Create Tables in Presto.

create external table if not exists default.sword_purchases (Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");

*Results:*

* *OK*
* *Time taken: 1.681 seconds*

*The remaining commands will have similar results.*

create external table if not exists default.knife_purchases (Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/knife_purchases'  tblproperties ("parquet.compress"="SNAPPY");

create external table if not exists default.guilds_joined (Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/guilds_joined'  tblproperties ("parquet.compress"="SNAPPY");

create external table if not exists default.sword_sold (Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/sword_sold'  tblproperties ("parquet.compress"="SNAPPY");

create external table if not exists default.knife_sold (Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/knife_sold'  tblproperties ("parquet.compress"="SNAPPY");

create external table if not exists default.guild_left (Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/guild_left'  tblproperties ("parquet.compress"="SNAPPY");

### Query Presto.

*In a separate terminal, we can query our data from Presto.*

docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

**presto:default> show tables;**

*Results:*
* *Table*      
* guild_left      
* guilds_joined   
* knife_purchases 
* knife_sold      
* sword_purchases 
* sword_sold      
* (6 rows)
* Query 20200730_034701_00002_2kv2a, FINISHED, 1 node
* Splits: 2 total, 1 done (50.00%)
* 0:01 [6 rows, 183B] [10 rows/s, 309B/s]


**presto:default> describe sword_purchases;**

*Results:*

* Column   |  Type   | Comment 
* ------------+---------+---------
*   accept     | varchar |         
*   host       | varchar |         
*   user_agent | varchar |         
*   event_type | varchar |         
*   timestamp  | varchar |         
*   raw_event  | varchar |         
*  (6 rows)
* Query 20200730_035412_00012_2kv2a, FINISHED, 1 node
* Splits: 2 total, 0 done (0.00%)
* 0:00 [6 rows, 446B] [13 rows/s, 998B/s]

**presto:default> select * from sword_purchases;**

*Results:*
* accept | host | user_agent | event_type | timestamp | raw_event 
* --------+------+------------+------------+-----------+-----------
* (0 rows)
* Query 20200730_035035_00011_2kv2a, FINISHED, 1 node
* Splits: 1 total, 0 done (0.00%)
* 0:03 [0 rows, 0B] [0 rows/s, 0B/s]

## Task 3: Use Apache Bench to generate test data for your pipeline

### Seed data into the Stream.

*In a separate terminal, generate events.*

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_knife

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_a_guild

*Results: (from Flask)*

* ...
* 127.0.0.1 - - [30/Jul/2020 17:31:37] "GET / HTTP/1.0" 200 -
* 127.0.0.1 - - [30/Jul/2020 17:31:45] "GET /purchase_a_sword HTTP/1.0" 200 -
* ...
* 127.0.0.1 - - [01/Aug/2020 01:07:45] "GET /purchase_a_knife HTTP/1.0" 200 -
* 127.0.0.1 - - [01/Aug/2020 01:08:01] "GET /join_a_guild HTTP/1.0" 200 -
* ...

*The remaining commands will have similar results.*

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/sell_a_sword

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/sell_a_knife

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/leave_a_guild

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_sword

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_knife

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/join_a_guild

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/sell_a_sword

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/sell_a_knife

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/leave_a_guild

### Query from Presto.

*In our previous Presto terminal, we can query again to see our newly generated data.*

**presto:default> select * from sword_purchases;**

*Results:*
* accept|        
* ----------------------------------------------------------------------------------------------------------+_
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user1.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* {"Host": "user2.att.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}     | 2020-07
* (20 rows)

### Feed the stream more data.

*In separate terminals, we can produce continuous streams of event data.*

while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword; sleep 10; done

while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_knife; sleep 10; done

while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_a_guild; sleep 10; done

while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/sell_a_sword; sleep 10; done

while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/sell_a_knife; sleep 10; done

while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/leave_a_guild; sleep 10; done

### Watch Presto grow.

**presto:default> select count(*) from sword_purchases;**

*Results:*
*  _col0 
*    140 
* (1 row)
* Query 20200730_174136_00008_q2w2x, FINISHED, 1 node
* Splits: 5 total, 1 done (20.00%)
* 0:01 [140 rows, 7.55KB] [97 rows/s, 5.28KB/s]

**presto:default> select count(*) from guilds_joined;**

*Results:*
*  _col0 
*    220 
* (1 row)
* Query 20200730_174253_00002_q2w2x, FINISHED, 1 node
* Splits: 5 total, 1 done (20.00%)
* 0:01 [140 rows, 7.55KB] [97 rows/s, 5.28KB/s]

### Check results in Hadoop.

*In a separate terminal, we can check our results.*

docker-compose exec cloudera hadoop fs -ls /tmp/sword_purchases

*Results:*
* Found 3 items
* drwxr-xr-x   - root supergroup          0 2020-07-30 18:06 /tmp/sword_purchases/_spark_metadata
* -rw-r--r--   1 root supergroup        688 2020-07-30 18:03 /tmp/sword_purchases/part-00000-1a11c64e-f43c-4781-a7fa-76c72c0109ad-c000.snappy.parquet
* -rw-r--r--   1 root supergroup       2409 2020-07-30 18:06 /tmp/sword_purchases/part-00000-88bedc7f-cee7-4969-a98c-585f06c16ade-c000.snappy.parquet

*The remaining commands will have similar results.*

docker-compose exec cloudera hadoop fs -ls /tmp/knife_purchases

docker-compose exec cloudera hadoop fs -ls /tmp/guilds_joined

docker-compose exec cloudera hadoop fs -ls /tmp/sword_sold

docker-compose exec cloudera hadoop fs -ls /tmp/knife_sold

docker-compose exec cloudera hadoop fs -ls /tmp/guild_left

## Task 4: Produce an analytics report where you provide a description of your pipeline and some basic analysis of the events

### Spin up Python Notebook.

docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

### Run Analytical Queries.

purchases = spark.read.parquet('/tmp/sword_purchases')

purchases.show()

*Results:*
* +--------------------+--------------------+------+-----------------+---------------+--------------+
* |           raw_event|           timestamp|Accept|             Host|     User-Agent|    event_type|
* +--------------------+--------------------+------+-----------------+---------------+--------------+
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* +--------------------+--------------------+------+-----------------+---------------+--------------+
* only showing top 20 rows

purchases.registerTempTable('purchases')

purchases_by_example2 = spark.sql("select * from purchases where host='user1.comcast.com'")

purchases_by_example2.show()

*Results:*
* +--------------------+--------------------+------+-----------------+---------------+--------------+
* |           raw_event|           timestamp|Accept|             Host|     User-Agent|    event_type|
* +--------------------+--------------------+------+-----------------+---------------+--------------+
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
* +--------------------+--------------------+------+-----------------+---------------+--------------+
* only showing top 20 rows

newdf = purchases_by_example2.toPandas()

newdf.describe()

*Results:*
* *raw_event*
* count : 40   
* unique : 1   
* top : {"Host": "user1.comcast.com", "event_type": "p...   
* freq  :40   


*  *timestamp*
* count : 40    
* unique : 40   
* top : 2020-07-30 18:06:28.696      
* freq : 1


* *Accept*   
* count : 40                
* unique : 1             
* top : */*    
* freq : 40


* *Host* 
* count : 40         
* unique :  1       
* top : user1.comcast.com  
* freq : 40     


* *User-Agent*
* count :  40   
* unique :  1   
* top  :  ApacheBench/2.3   
* freq : 40   


* *event_type*  
* count :40  
* unique : 1  
* top :  purchase_sword  
* freq  : 40  


my_guilds = spark.read.parquet('/tmp/guilds_joined')

my_guilds.show()

*Results:*
* +--------------------+--------------------+------+-----------------+---------------+----------+
* |           raw_event|           timestamp|Accept|             Host|     User-Agent|event_type|
* +--------------------+--------------------+------+-----------------+---------------+----------+
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* |{"Host": "user1.c...|2020-07-30 18:06:...|   */*|user1.comcast.com|ApacheBench/2.3|join_guild|
* +--------------------+--------------------+------+-----------------+---------------+----------+
* only showing top 20 rows

guilds.registerTempTable('my_guilds')

guilds_by_example = spark.sql("select * from purchases where host='user2.att.com'")

guilds_by_example.show()

*Results:*
* +--------------------+--------------------+------+-------------+---------------+----------+
* |           raw_event|           timestamp|Accept|         Host|     User-Agent|event_type|
* +--------------------+--------------------+------+-------------+---------------+----------+
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* |{"Host": "user2.a...|2020-07-30 18:06:...|   */*|user2.att.com|ApacheBench/2.3|join_guild|
* +--------------------+--------------------+------+-------------+---------------+----------+



### Clean up.

docker-compose down

*Results:*
* Stopping project3etwernermids_spark_1 ... done
* Stopping project3etwernermids_kafka_1 ... done
* Stopping project3etwernermids_zookeeper_1 ... done
* Stopping project3etwernermids_mids_1 ... done
* Stopping project3etwernermids_presto_1 ... done
* Stopping project3etwernermids_cloudera_1 ... done
* Removing project3etwernermids_spark_1 ... done
* Removing project3etwernermids_kafka_1 ... done
* Removing project3etwernermids_zookeeper_1 ... done
* Removing project3etwernermids_mids_1 ... done
* Removing project3etwernermids_presto_1 ... done
* Removing project3etwernermids_cloudera_1 ... done
* Removing network project3etwernermids_default